In [ ]:
Preprocessing : 전처리
Tokenizing => Tokens = {음절, 어절, 단어, 형태소, 어근, 접사, 어간, 어미, 구 ... ngram, 품사}
           => 왜? => 좋은 feature들을 찾으려고. => 좋은 토큰을 뽑아야지만, 좋은 모델이 만들어진다.

zip's law => 빈도의 순으로 나열 = 순위의 역순이랑 동일, 가장 많이 나온 단어는 그 다음 단어의 두배
             고빈도단어(문장부호, 접사, 조사.... -> 의미 파악시 별로 안 중요함); 상위 n개의 단어가 대부분 차지
             저빈도단어(고유명사, 신조어,비속어,오탈자); 토큰 후보들 중 대부분을 차지하는 너무 많음
Heaps law =>  문서에 나타난 고유한 단어들은 전체 단어의 수와 일전한 관계

tokenizing 기법:
NLTK -> sent_tokenize : 문장 단위로 쪼갤 때 사용한다. , word_tokenize => regex_tokenize, Tweettokenize
한글  -> Morpheme analyser, POS, nouns .... 품사표(tagset)
        => key 못 찾음; Out of Vocabulary (OOV) =>unknown 토은 저장하면된데, 이제 근간이 되는 데이커기때문에...

Entropy, Perplexity(Cohestipon) => 조건부확률
Stemming. (Stem, 어간); 어간+어미; 있다| 있었다| 있었고 | 있었으며 .. 
Lemmatization(Lemma) 페제어 추출; 어근/원형, 날다 + 나/는 ... => 날/눈,,,,,
                           is  was be ... -:

SPM - WPM - BPE(*)   => 워드피스모델: 단어조각
; Sententce-picec Model(문장; 분절 띄어쓰기)=> 0
      ; Word-Piece-Model(어절;분절)
            ; Byte Pair Encoding 방법 (*) 

Normaliazion
대소문자, 약어(LA, losangeles), Stopwords

In [ ]:
from konlpy.tag import Hannanum

In [ ]:
ma = Hannanum()
data='''2022년 말 GTP-3.5 기반의 챗GPT를 공개한 오픈AI는 불과 4개월 만인 지난해 3월 GPT-4를 선보였다. 매개변수는 공개하지 않았지만 1750억개인 GPT-3.5보다 증가했고 인식 가능한 명령어도 GPT-3.5가 문자만 가능했다면 GPT-4는 문자는 물론 이미지까지 포함했다.

인식 가능한 글 길이도 단어 약 3000개에서 2만5000여 개로 늘어났다. 오픈AI는 GPT-3.5의 미국 변호사 시험 성적이 하위 10%였다면 GPT-4는 상위 10%를 기록하면서 보다 인간다워졌다고 밝혔다. 그리고 지난해 11월에는 멀티모달리티 기능을 강화한 'GPT-4 터보'를 선보인 바 있다.

업계는 오픈AI가 공개를 앞둔 GPT-5가 범용 인공지능(AGI)에 얼마나 가까워졌을지 주목하고 있다. 아직 완벽한 AGI 구현은 어렵겠지만 GPT-5가 AGI를 개발하는 데 징검다리 역할을 할 수 있기 때문이다.
'''


In [ ]:
ma.nouns(data)

In [ ]:
from konlpy.tag import Hannanum, Kkma, Komoran, Okt

ma = [Hannanum(), Kkma(), Komoran(), Okt()]

In [ ]:
for m in ma:
    print(m.pos('하늘을 나는 새'))

# 띄어쓰기

In [ ]:
pwd

In [ ]:
# 뉴스 수집기

In [ ]:
from requests.compat import urljoin
from requests import get
from bs4 import BeautifulSoup
import re

urls = ['https://news.naver.com']
seens = []
path = './news'

while urls:
    url = urls.pop(0)
    
    resp = get(url)
    seens.append(url)

    if not re.search(r'text\/html', resp.headers['content-type']):
        continue
        
    dom = BeautifulSoup(resp.text, 'html.parser')
    
    # 1번 뉴스 메뉴
    for a in dom.select('.Nlnb_menu_list > li > a[href]')[1:7]:
        nurl = urljoin(url, a.attrs['href'])
        if nurl not in urls and nurl not in seens:
            urls.append(nurl)
    # 헤드라인
    for a in dom.select('[id^="_SECTION_HEADLINE_LIST"] .sa_text_title[href]'):
        nurl = urljoin(url, a.attrs['href'])
        if nurl not in urls and nurl not in seens:
            urls.append(nurl)

    # 3번 - 뉴스본문 # 원래는 있는지 없는지 봐야함
    c = dom.select_one('#contents')  
    if c: 
        file = re.search(r'(\d{8,})$',url).group(1)
        with open(f'{path}{file}.txt', 'w', encoding='utf8') as f:
            f.write(c.get_text())

In [6]:
from os import listdir
def fileids(path):
    return list(map(lambda f:path + ('' if path[-1] == '/' else '/')+f, listdir(path)))
    
fileids('./')

['./Untitled Folder',
 './0325.ipynb',
 './0327.ipynb',
 './0326.ipynb',
 './news',
 './.ipynb_checkpoints']

In [ ]:
ls

In [8]:
import re
corpus = list()
for file in fileids('news'):
    with open(fileids('./news')[0], 'r', encoding='utf8') as f:
        d = f.read()
        corpus.append(  re.sub( r'\sCopyright.+', '', re.sub( r'^\s+|\s+$', '', re.sub( r'\s+', ' ', re.sub(r'\x00-\xff', '', d))))  )

In [9]:
from nltk.text import Text
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer  # 어간 추출, 어간: 단어의 줄기
import matplotlib as plt

In [10]:
PorterStemmer().stem('play'),\
PorterStemmer().stem('played'),\
PorterStemmer().stem('playing'),\
PorterStemmer().stem('was')  # ==> 이렇게 오류가 나는 지점 때문에 lemma도 필요하다

('play', 'play', 'play', 'wa')

In [11]:
tObj = list()

In [ ]:
from konlpy.tag import Hannanum, Kkma, Komoran, Okt

ma = [Hannanum(), Kkma(), Komoran(), Okt()]

In [ ]:
for d in corpus:
    if len(tObj) == 0:
        tObj.append(Text(word_tokenize(d)).vocab)
    else:
        tObj.append(tObj[-1] + Text(word_tokenize(d)).vocab())
print(tokens)

In [ ]:
tObj[-1].N(), tObj[-1].B()

In [ ]:
kkmtObj = list()

for d in corpus:
    if len(kkmtObj) == 0:
        kkmtObj.append(Text(ma[1].morphs(d)).vocab())
    else:
        kkmtObj.append(kkmObj[-1] + Text(ma[1].morphs(d)).vocab())

In [ ]:
# zips laws
n = 50
plt.plot([1/i for i in range(1,n+1)], c='k')
plt.plot([r[1]/tObj[-1].get(tObj[-1].max()) for r in tObj[-1].most_common(n)], c='k')

In [ ]:
# heaps laws
k = 12
b = .48
plt.plot([k.B() for k in kkmtObj], c='k')
plt.plot([12*k.N() for k in kkmtObj], c='r')

In [13]:
def ngram(s,n=2,t=True):  #음절이냐, 어절이냐의 차이로 해본다.  # t=True; 어절  t=False; 음절
    result  = []
    if not t:
        s = list(s)

    for i in range(len(s)-(n-1)):
        result.append(tuple(s[i:i+n]))
        
    return result

In [14]:
gram = {'1': ngram()}

TypeError: ngram() missing 1 required positional argument: 's'

In [15]:
gram1 = list()
gram2 = list()
gram3 = list()
gram4 = list()

for c in corpus:
    gram1.exend(ngram(c,t=False))
    gram2.exend(ngram(c,t=False))
    gram3.exend(ngram(c,t=False))
    gram4.exend(ngram(c,t=False))

NameError: name 'corpus' is not defined

In [16]:
from collections import Counter

In [17]:
c1 = Counter(gram1)
c2 = Counter(gram2)
c3 = Counter(gram3)
c4 = Counter(gram4)

In [18]:
c4

Counter()

In [28]:
n = 50
plt.plot([1/i for i in range(1,n+1)], c='k')
plt.plot([i[1]/max(c1.values()) for i in c1.most_common(n)], c='k')
plt.plot([i[2]/max(c2.values()) for i in c2.most_common(n)], c='r')
plt.plot([i[3]/max(c3.values()) for i in c3.most_common(n)], c='g')
plt.plot([i[4]/max(c4.values()) for i in c4.most_common(n)])

NameError: name 'plt' is not defined

In [29]:
from nltk.tokenize import sent_tokenize

re.sub(r'\s', '', sent_tokenize(corpus([0])[0])

SyntaxError: incomplete input (2043153614.py, line 3)

In [30]:
fileids('news')[0]

NameError: name 'fileids' is not defined

In [ ]:
q = re.sub(r'\s',)

In [31]:
corpus

NameError: name 'corpus' is not defined

In [32]:
def findkey(gram, key):
    k = tuple(key)
    return filter(lambdal g:g[:len(k)], )

findKey(gram2),'가')


SyntaxError: unmatched ')' (1918617479.py, line 5)